In [1]:
# !pip install swifter
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, make_scorer
from IPython.display import FileLink
from tqdm import tqdm_notebook 
from tqdm.notebook import tqdm
import lightgbm as lgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
import warnings
# import swifter
from joblib import Parallel, delayed
from tsfresh.utilities.dataframe_functions import roll_time_series, make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
train_file = '../input/df-exploded/result_{}.csv'
super_train = '../input/df-exploded/super_train.csv'


In [3]:
stock = 1
df = pd.read_csv(train_file.format(stock), index_col='ID', parse_dates=['Date'])
    
df

,stock,unpredictability_score,Close,High,Low,Open,holiday,Date,Open_upper__benford_correlation,Open_hat__benford_correlation,...,Close_trend__benford_correlation,Open_trend__benford_correlation,Open_ trend_upper__benford_correlation,Close_ trend_upper__benford_correlation,Close_trend_lower__benford_correlation,High_lower__benford_correlation,High_trend__benford_correlation,High_upper__benford_correlation,High_trend_lower__benford_correlation,High_ trend_upper__benford_correlation
ID,,,,,,,,,,,,,,,,,,,,,
id_754,1,0,38.4540,38.2343,38.6799,38.4097,1,2017-01-03,0.062915,0.062915,...,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915
id_755,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-04,0.062915,0.062915,...,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915
id_756,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-05,0.062915,0.062915,...,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915
id_757,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-06,0.062915,0.062915,...,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915
id_758,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-09,0.062915,0.062915,...,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_1503,1,0,NaN,NaN,NaN,NaN,0,2019-12-24,-0.064614,-0.064614,...,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614
id_1504,1,0,NaN,NaN,NaN,NaN,1,2019-12-26,-0.064614,-0.064614,...,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614
id_1505,1,0,NaN,NaN,NaN,NaN,0,2019-12-27,-0.064614,-0.064614,...,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614


In [4]:
tdf = pd.read_csv(super_train, index_col='ID')

tdf

,Open_hat,Open_trend,Open_lower,Open_upper,Open_trend_lower,Open_ trend_upper,High_hat,High_trend,High_lower,High_upper,...,Open,High,Low,Close,holiday,unpredictability_score,Open_diff,Low_diff,High_diff,Close_diff
ID,,,,,,,,,,,,,,,,,,,,,
id_0,81.025967,82.386498,78.803918,83.056482,82.386498,82.386498,81.202899,82.597286,79.340480,83.157889,...,82.9961,82.7396,82.9144,82.8101,1,7,1.970133,2.002479,1.536701,1.975953
id_1,80.865875,82.437997,78.822365,82.974286,82.437997,82.437997,81.046504,82.643819,79.061205,83.051594,...,83.1312,83.1669,83.3779,82.9690,0,7,2.265325,2.466027,2.120396,2.151123
id_2,80.976781,82.489497,78.898351,83.082205,82.489497,82.489497,81.080195,82.690352,79.154399,83.025130,...,82.6622,82.7634,82.8984,82.8578,0,7,1.685419,1.803884,1.683205,1.912449
id_3,81.060195,82.540996,78.763372,83.062250,82.540996,82.540996,81.148817,82.736884,79.077222,83.003408,...,83.0279,82.7950,82.8425,82.7385,0,7,1.967705,1.628570,1.646183,1.651943
id_4,81.141042,82.695494,79.101468,83.174879,82.695494,82.695494,81.171321,82.876482,78.954744,83.127788,...,82.3761,82.0828,82.1473,81.8641,0,7,1.235058,0.733757,0.911479,0.672735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_77657,133.246558,130.185331,130.398763,135.963138,128.413669,131.840591,132.715079,129.880891,129.605239,135.501505,...,NaN,NaN,NaN,NaN,0,6,0.153895,0.051313,0.258674,0.197796
id_77658,133.613717,130.157991,130.816426,136.724601,128.277605,131.948423,133.031871,129.850858,129.984751,135.854295,...,NaN,NaN,NaN,NaN,1,6,0.457144,0.291064,0.598355,0.414344
id_77659,133.402365,130.144321,130.475015,136.215723,128.193173,132.004618,132.892882,129.835841,129.924311,135.739193,...,NaN,NaN,NaN,NaN,0,6,0.152253,0.143879,0.246128,0.265842


In [5]:
# print([i for i in df.columns if 'Open' == i])
# print([i for i in df.columns if 'High' == i])
# print([i for i in df.columns if 'Low' == i])
# print([i for i in df.columns if 'holiday' == i])





In [6]:
df = df.join(tdf[['Open_hat', 'High_hat', 'Low_hat', 'Close_hat']])

In [7]:
def expand_df(dframe):
    dFrame = dframe.copy()
    dFrame['day'] = dFrame.Date.apply(lambda x: x.day)
    dFrame['month'] = dFrame.Date.apply(lambda x: x.month)
    dFrame['year'] = dFrame.Date.apply(lambda x: x.year)
    dFrame['dayofweek'] = dFrame.Date.apply(lambda x: x.dayofweek)
    dFrame['dayofyear'] = dFrame.Date.apply(lambda x: x.dayofyear)
    dFrame['weekofyear'] = dFrame.Date.apply(lambda x: x.weekofyear)
    dFrame['year_diff'] = dFrame.Date.apply(lambda x: x.year - 2017)
    dFrame['days_so_far_skipped'] = dFrame.Date.apply(lambda x: dFrame[dFrame.Date < x].shape[0])
    dFrame['days_so_far'] = dFrame.Date.apply(lambda x: (x - pd.Timestamp('2017-01-03')).days)
    return dFrame

In [8]:
df = expand_df(df)

In [9]:
# for col in df.columns:
#     if df[col].isna().sum() != 0: print(col)

In [10]:
cat_cols = [ 
    'holiday',
    'stock',
    'day',
     'month',
     'year',
     'dayofweek',
     'dayofyear',
     'weekofyear',
    'year_diff', 
    'unpredictability_score']
excluded_cols = ['Close_hat', 'Open_hat', 'High_hat', 'Low_hat']
                 

In [11]:
def get_rolling_mean(dframe, col, idx, days=30, met='mean'):
#     print(col, idx, days, met)
    if met == 'mean':
        return dframe[(dframe['days_so_far_skipped'] < dframe.loc[idx]['days_so_far_skipped'] + days) & (dframe['days_so_far_skipped'] >= dframe.loc[idx]['days_so_far_skipped'])][col].mean()
    if met == 'max':
        return dframe[(dframe['days_so_far_skipped'] < dframe.loc[idx]['days_so_far_skipped'] + days) & (dframe['days_so_far_skipped'] >= dframe.loc[idx]['days_so_far_skipped'])][col].max()
    return dframe[(dframe['days_so_far_skipped'] < dframe.loc[idx]['days_so_far_skipped'] + days) & (dframe['days_so_far_skipped'] >= dframe.loc[idx]['days_so_far_skipped'])][col].min()

def rolled_mean(dframe, timeshift=30):
    dframe['ID'] = dframe.index

    for col in excluded_cols:
        dframe[col + '_roll_mean_per_mon'] = dframe['ID'].apply(lambda x: get_rolling_mean(dframe, col, x, days=timeshift, met='mean'))
        dframe[col + '_roll_max_per_mon'] = dframe['ID'].apply(lambda x: get_rolling_mean(dframe, col, x, days=timeshift, met='max'))
        dframe[col + '_roll_min_per_mon'] = dframe['ID'].apply(lambda x:get_rolling_mean(dframe, col, x, days=timeshift, met='min'))
        dframe[col + '_roll_range_per_mon'] = dframe[col + '_roll_max_per_mon'] - dframe[col + '_roll_min_per_mon']
        dframe[col + '_roll_mean_per_d'] = dframe['ID'].apply(lambda x: get_rolling_mean(dframe, col, x, days=2, met='mean'))
        dframe[col + '_roll_max_per_d'] = dframe['ID'].apply(lambda x: get_rolling_mean(dframe, col, x, days=2, met='max'))
        dframe[col + '_roll_min_per_d'] = dframe['ID'].apply(lambda x:get_rolling_mean(dframe, col, x, days=2, met='min'))
        dframe[col + '_roll_range_per_d'] = dframe[col + '_roll_max_per_d'] - dframe[col + '_roll_min_per_d']
    
    return dframe.set_index('ID', drop=True)

In [12]:
# dx = rolled_mean(df)
# best_d, idx = 0, 0
# for i in df.index:
#     d = dx[dx['ID'] == i]['Close_hat'].mean() - df.loc[i]['Close_hat']
#     if abs(d) > best_d:
#         best_d = abs(d)
        
#         idx = i
# #     print(dx[dx['ID'] == i]['Close_hat'].shape)#.eq(df.loc[i]['Close_hat']), i)
    
# print(best_d)

In [13]:
df

,stock,unpredictability_score,Close,High,Low,Open,holiday,Date,Open_upper__benford_correlation,Open_hat__benford_correlation,...,Close_hat,day,month,year,dayofweek,dayofyear,weekofyear,year_diff,days_so_far_skipped,days_so_far
ID,,,,,,,,,,,,,,,,,,,,,
id_754,1,0,38.4540,38.2343,38.6799,38.4097,1,2017-01-03,0.062915,0.062915,...,38.243238,3,1,2017,1,3,1,0,0,0
id_755,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-04,0.062915,0.062915,...,38.298977,4,1,2017,2,4,1,0,1,1
id_756,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-05,0.062915,0.062915,...,38.334698,5,1,2017,3,5,1,0,2,2
id_757,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-06,0.062915,0.062915,...,38.381561,6,1,2017,4,6,1,0,3,3
id_758,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-09,0.062915,0.062915,...,38.520599,9,1,2017,0,9,2,0,4,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_1503,1,0,NaN,NaN,NaN,NaN,0,2019-12-24,-0.064614,-0.064614,...,44.560011,24,12,2019,1,358,52,2,749,1085
id_1504,1,0,NaN,NaN,NaN,NaN,1,2019-12-26,-0.064614,-0.064614,...,44.515129,26,12,2019,3,360,52,2,750,1087
id_1505,1,0,NaN,NaN,NaN,NaN,0,2019-12-27,-0.064614,-0.064614,...,44.535579,27,12,2019,4,361,52,2,751,1088


In [14]:

df = rolled_mean(df)
df

,stock,unpredictability_score,Close,High,Low,Open,holiday,Date,Open_upper__benford_correlation,Open_hat__benford_correlation,...,High_hat_roll_min_per_d,High_hat_roll_range_per_d,Low_hat_roll_mean_per_mon,Low_hat_roll_max_per_mon,Low_hat_roll_min_per_mon,Low_hat_roll_range_per_mon,Low_hat_roll_mean_per_d,Low_hat_roll_max_per_d,Low_hat_roll_min_per_d,Low_hat_roll_range_per_d
ID,,,,,,,,,,,,,,,,,,,,,
id_754,1,0,38.4540,38.2343,38.6799,38.4097,1,2017-01-03,0.062915,0.062915,...,38.040274,0.039247,38.824593,39.048402,38.501096,0.547305,38.528666,38.556235,38.501096,0.055138
id_755,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-04,0.062915,0.062915,...,38.079520,0.033626,38.833514,39.048402,38.556235,0.492167,38.579376,38.602518,38.556235,0.046283
id_756,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-05,0.062915,0.062915,...,38.113146,0.051561,38.841296,39.048402,38.602518,0.445884,38.619642,38.636765,38.602518,0.034247
id_757,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-06,0.062915,0.062915,...,38.164707,0.140384,38.847847,39.048402,38.636765,0.411637,38.708924,38.781083,38.636765,0.144318
id_758,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-09,0.062915,0.062915,...,38.305092,0.080633,38.857223,39.048402,38.694696,0.353706,38.812747,38.844411,38.781083,0.063328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_1503,1,0,NaN,NaN,NaN,NaN,0,2019-12-24,-0.064614,-0.064614,...,44.218926,0.069086,44.762568,44.823390,44.714463,0.108926,44.739775,44.765087,44.714463,0.050623
id_1504,1,0,NaN,NaN,NaN,NaN,1,2019-12-26,-0.064614,-0.064614,...,44.218926,0.030340,44.761938,44.823390,44.714463,0.108926,44.722756,44.731048,44.714463,0.016584
id_1505,1,0,NaN,NaN,NaN,NaN,0,2019-12-27,-0.064614,-0.064614,...,44.249267,0.044590,44.777763,44.823390,44.731048,0.092342,44.754949,44.778850,44.731048,0.047802


In [15]:
# df = rolled_mean(df)
df.shape

(754, 557)

In [16]:
df['Close_hat'].equals(df['Close_hat_roll_mean_per_mon'])

False

In [17]:
sum(df['Close_hat'] - df['Close_hat_roll_mean_per_mon'])

-87.72633256001151

In [18]:
sum(df[df['Close'].notna()]['Close'] - df[df['Close'].notna()]['Close_hat'])

-0.0021594698716782545

In [19]:
[col for col in tdf.columns.tolist() if col in set(excluded_cols)]

['Open_hat', 'High_hat', 'Low_hat', 'Close_hat']

In [20]:

# ['mean', 'maximum', 'minimum']
df.columns.tolist()

['stock',
 'unpredictability_score',
 'Close',
 'High',
 'Low',
 'Open',
 'holiday',
 'Date',
 'Open_upper__benford_correlation',
 'Open_hat__benford_correlation',
 'Low_hat__benford_correlation',
 'Low_trend__benford_correlation',
 'Low_trend_lower__benford_correlation',
 'Low_ trend_upper__benford_correlation',
 'Close_upper__benford_correlation',
 'Open_hat__fft_coefficient__attr_"abs"__coeff_0',
 'Open_hat__fft_coefficient__attr_"real"__coeff_0',
 'Open_hat__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)',
 'Open_hat__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)',
 'Open_hat__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)',
 'Open_hat__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)',
 'Open_hat__sum_values',
 'Open_hat__quantile__q_0.8',
 'Open_hat__abs_energy',
 'Open_hat__median',
 'Open_hat__mean',
 'Open_hat__quantile__q_0.9',
 'Open_hat__minimum',
 'Open_hat__maximum',
 'Open_hat__quantile__q_0.2',
 'Open_hat__quantile__q_0.3',
 'Open_hat__quant

In [21]:
tdf

,Open_hat,Open_trend,Open_lower,Open_upper,Open_trend_lower,Open_ trend_upper,High_hat,High_trend,High_lower,High_upper,...,Open,High,Low,Close,holiday,unpredictability_score,Open_diff,Low_diff,High_diff,Close_diff
ID,,,,,,,,,,,,,,,,,,,,,
id_0,81.025967,82.386498,78.803918,83.056482,82.386498,82.386498,81.202899,82.597286,79.340480,83.157889,...,82.9961,82.7396,82.9144,82.8101,1,7,1.970133,2.002479,1.536701,1.975953
id_1,80.865875,82.437997,78.822365,82.974286,82.437997,82.437997,81.046504,82.643819,79.061205,83.051594,...,83.1312,83.1669,83.3779,82.9690,0,7,2.265325,2.466027,2.120396,2.151123
id_2,80.976781,82.489497,78.898351,83.082205,82.489497,82.489497,81.080195,82.690352,79.154399,83.025130,...,82.6622,82.7634,82.8984,82.8578,0,7,1.685419,1.803884,1.683205,1.912449
id_3,81.060195,82.540996,78.763372,83.062250,82.540996,82.540996,81.148817,82.736884,79.077222,83.003408,...,83.0279,82.7950,82.8425,82.7385,0,7,1.967705,1.628570,1.646183,1.651943
id_4,81.141042,82.695494,79.101468,83.174879,82.695494,82.695494,81.171321,82.876482,78.954744,83.127788,...,82.3761,82.0828,82.1473,81.8641,0,7,1.235058,0.733757,0.911479,0.672735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_77657,133.246558,130.185331,130.398763,135.963138,128.413669,131.840591,132.715079,129.880891,129.605239,135.501505,...,NaN,NaN,NaN,NaN,0,6,0.153895,0.051313,0.258674,0.197796
id_77658,133.613717,130.157991,130.816426,136.724601,128.277605,131.948423,133.031871,129.850858,129.984751,135.854295,...,NaN,NaN,NaN,NaN,1,6,0.457144,0.291064,0.598355,0.414344
id_77659,133.402365,130.144321,130.475015,136.215723,128.193173,132.004618,132.892882,129.835841,129.924311,135.739193,...,NaN,NaN,NaN,NaN,0,6,0.152253,0.143879,0.246128,0.265842


In [22]:
encoder = LabelEncoder()
for col in tqdm(cat_cols):
    df[col] = encoder.fit_transform(df[col])

In [23]:
# df['train'] = df['Close'].apply(lambda x: not pd.isna(x))
# df.columns = [i.replace('{', '_').replace('}', '_') for i in df.columns]
df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]

In [24]:
X, y = df[df['Close'].notna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].notna()]
X_test, y_test = df[df['Close'].isna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].isna()]

In [25]:
X['days_so_far']

ID
id_754        0
id_755        1
id_756        2
id_757        3
id_758        6
           ... 
id_1462    1025
id_1463    1028
id_1464    1029
id_1465    1030
id_1466    1031
Name: days_so_far, Length: 713, dtype: int64

In [26]:
X

,stock,unpredictability_score,holiday,Date,Open_upper__benford_correlation,Open_hat__benford_correlation,Low_hat__benford_correlation,Low_trend__benford_correlation,Low_trend_lower__benford_correlation,Low__trend_upper__benford_correlation,...,High_hat_roll_min_per_d,High_hat_roll_range_per_d,Low_hat_roll_mean_per_mon,Low_hat_roll_max_per_mon,Low_hat_roll_min_per_mon,Low_hat_roll_range_per_mon,Low_hat_roll_mean_per_d,Low_hat_roll_max_per_d,Low_hat_roll_min_per_d,Low_hat_roll_range_per_d
ID,,,,,,,,,,,,,,,,,,,,,
id_754,0,0,1,2017-01-03,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,...,38.040274,0.039247,38.824593,39.048402,38.501096,0.547305,38.528666,38.556235,38.501096,0.055138
id_755,0,0,0,2017-01-04,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,...,38.079520,0.033626,38.833514,39.048402,38.556235,0.492167,38.579376,38.602518,38.556235,0.046283
id_756,0,0,0,2017-01-05,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,...,38.113146,0.051561,38.841296,39.048402,38.602518,0.445884,38.619642,38.636765,38.602518,0.034247
id_757,0,0,0,2017-01-06,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,...,38.164707,0.140384,38.847847,39.048402,38.636765,0.411637,38.708924,38.781083,38.636765,0.144318
id_758,0,0,0,2017-01-09,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,...,38.305092,0.080633,38.857223,39.048402,38.694696,0.353706,38.812747,38.844411,38.781083,0.063328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_1462,0,0,0,2019-10-25,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,...,45.420615,0.082395,45.487828,45.906257,45.287613,0.618644,45.882964,45.906257,45.859671,0.046587
id_1463,0,0,0,2019-10-28,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,...,45.420615,0.003451,45.464898,45.859671,45.218351,0.641320,45.858177,45.859671,45.856682,0.002989
id_1464,0,0,0,2019-10-29,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,...,45.361671,0.062395,45.443148,45.856682,45.207164,0.649518,45.836807,45.856682,45.816932,0.039750


In [27]:
y

,Close,stock
ID,,
id_754,38.4540,0
id_755,38.5994,0
id_756,38.5994,0
id_757,38.5994,0
id_758,38.5994,0
...,...,...
id_1462,45.9002,0
id_1463,45.9425,0
id_1464,45.9515,0


In [28]:
# hyper_params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'regression',
#     'metric': ['rmse'],
#     'learning_rate': 0.1,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.7,
#     'bagging_freq': 10,
#     'verbose': 0,
#     "max_depth": 4,
#     "num_leaves": 128,  
#     "max_bin": 512,
#     "num_iterations": 100000,
#     "n_estimators": 1000,
#     "random_state": 32
# }
# model_store = [0] * 103
# metrics = [0] * 103
# preds = []
# # for stock in tqdm(X.stock.unique(), total=103):
# for stock in tqdm(range(103), total=103):
# # for stock in tqdm([1]):
#     df = pd.read_csv(train_file.format(stock), index_col='ID', parse_dates=['Date'])
#     df = expand_df(df)
#     encoder = LabelEncoder()
#     for col in cat_cols:
#         df[col] = encoder.fit_transform(df[col])
        
#     df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]
#     X, y = df[df['Close'].notna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].notna()]
#     X_test, y_test = df[df['Close'].isna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].isna()]
    
    
#     X_tr, X_val, y_tr, y_val = train_test_split(X.drop(columns=['Date']), y['Close'], train_size=0.80, random_state=11568)
    
    
#     model_store[stock] = lgb.LGBMRegressor(**hyper_params)
#     model_store[stock].fit(X_tr, 
#                            y_tr,
#                            eval_set=[(X_val, y_val)],
#                            eval_metric='rmse',
#                            early_stopping_rounds=100,
#                            verbose=False,
#                            )
#     metrics[stock] = list(list(model_store[stock].best_score_.values())[0].values())[0]
#     print(stock, metrics[stock])
#     preds.append(pd.DataFrame({'ID': X_test.index, 'Close': model_store[stock].predict(X_test.drop(columns=['Date']), num_iteration=model_store[stock].best_iteration_)}))



# # pd.concat(preds).to_csv('result.csv', index=False)
# # FileLink('result.csv')

In [29]:
def rmse(y_true, y_pred):
    return abs(mean_squared_error(y_true, y_pred, squared=False))

In [30]:
model_store1 = [0] * 103
metrics1 = [0] * 103
# df = None

params = {'learning_rate': [.35, .40, .45], 'depth': [1, 2, 3, 4]}


preds1 = []

def get_predictions(stock):
    df = pd.read_csv(train_file.format(stock), index_col='ID', parse_dates=['Date'])
    df = df.join(tdf[['Open_hat', 'High_hat', 'Low_hat', 'Close_hat']])
    df = expand_df(df)

    df = rolled_mean(df)

    encoder = LabelEncoder()
    for col in cat_cols:
        df[col] = encoder.fit_transform(df[col])

    df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]

    X, y = df[df['Close'].notna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].notna()]
    X_test, y_test = df[df['Close'].isna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].isna()]

    X_tr, X_val, y_tr, y_val = train_test_split(X, y['Close'], train_size=.8, random_state=11568)
    model = CatBoostRegressor(
        loss_function='RMSE', 
#         depth=2, 
#         learning_rate=0.4, 
        iterations=800,
        random_seed=18,
        od_type='Iter',
        od_wait=20,
        thread_count=1, task_type="GPU"
    )
    
#     model_store1[stock].fit(
#         X_tr.drop('Date', axis=1), y_tr, use_best_model=True,
#         cat_features=cat_cols,
#         eval_set=(X_val.drop('Date', axis=1), y_val),
#         verbose=False,
#         plot=False,
#     )
    grid = GridSearchCV(estimator=model, param_grid=params, cv=2, n_jobs=1, 
                        scoring=make_scorer(rmse, greater_is_better=False), verbose=0)
    grid.fit(
        X.drop('Date', axis=1), y['Close']
    )
# #     model_store1[stock] = grid
#     print("best_params")
#     metrics1[stock] = grid.best_params_
    return pd.DataFrame({'Params': str(grid.best_params_), 'Score': abs(grid.best_score_), 'ID': X_test.index, 'Close': grid.predict(X_test.drop('Date', axis=1))})


num_cores = 8
# preds1 = Parallel(n_jobs=num_cores)(delayed(get_predictions)(stock) for stock in tqdm(range(1)))
# preds1 = [get_predictions(stock) for stock in tqdm(range(103))]

In [31]:
# preds1[0]

In [ ]:
model_store = [0] * 103
metrics = [0] * 103

preds = []

parameters = {#'nthread':[1], #when use hyperthread, xgboost may become slower
              'objective':['reg:squarederror'],
              'learning_rate': [.1, .2, .3], #so called `eta` value
              'max_depth': [7, 8, 6],
#               'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7, .9],
#               'colsample_bytree': [0.7],
              'n_estimators': [500, 1000],
                'eval_metric': ['rmse']}


def get_predictions1(stock):
    df = pd.read_csv(train_file.format(stock), index_col='ID', parse_dates=['Date'])
    df = df.join(tdf[['Open_hat', 'High_hat', 'Low_hat', 'Close_hat']])
    df = expand_df(df)

    df = rolled_mean(df)

    encoder = LabelEncoder()
    for col in cat_cols:
        df[col] = encoder.fit_transform(df[col])

    df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]

    X, y = df[df['Close'].notna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].notna()]
    X_test, y_test = df[df['Close'].isna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].isna()]

    X_tr, X_val, y_tr, y_val = train_test_split(X, y['Close'], train_size=.8, random_state=11568)
    model = XGBRegressor(tree_method='gpu_hist', gpu_id=0)
    
    xgb_grid = GridSearchCV(model,
                           parameters,
                           cv=2,
                           n_jobs=1,
                           scoring=make_scorer(rmse, greater_is_better=False),
                           verbose=False
                           )
    
    xgb_grid.fit(
        X.drop(columns=['Date', 'unpredictability_score']), y['Close'],
        
    )
#     model_store[stock] = xgb_grid
#     metrics[stock] = xgb_grid.best_score_
#     print(str(xgb_grid.best_params_))
    return pd.DataFrame({'Params': str(xgb_grid.best_params_), 'Score': abs(xgb_grid.best_score_), 'ID': X_test.index, 'Close': xgb_grid.predict(X_test.drop(columns=['Date', 'unpredictability_score']))})


num_cores = 4
preds = Parallel(n_jobs=num_cores)(delayed(get_predictions1)(stock) for stock in tqdm(range(103)))

# preds = [get_predictions1(stock) for stock in tqdm(range(1))]

In [ ]:
preds[0]

In [ ]:
# preds = []
# for stock in X.stock.unique(): 

# # for stock in [1]:
#     pred1 = model_store[stock].predict(X_test[X_test['stock'] == stock].drop(columns=['Date']), num_iteration=model_store[stock].best_iteration_)
#     pred2 = model_store1[stock].predict(X_test[X_test['stock'] == stock])
    
#     if metrics[stock] > metrics1[stock]:
#         print("picked first")
# #         preds.append(pd.DataFrame({'ID': X_test[X_test['stock'] == stock].index, 'Close': model_store1[stock].predict(X_test[X_test['stock'] == stock])}))
#         preds.append(pd.DataFrame({'ID': X_test[X_test['stock'] == stock].index, 'Close': (pred2 * 7 + pred1) / 8}))
    
#     else:
#         print("picked second")
# #         preds.append(pd.DataFrame({'ID': X_test[X_test['stock'] == stock].index, 'Close': model_store[stock].predict(X_test[X_test['stock'] == stock].drop(columns=['Date']), num_iteration=model_store[stock].best_iteration_)}))
#         preds.append(pd.DataFrame({'ID': X_test[X_test['stock'] == stock].index, 'Close': (pred2 * 4 + pred1) / 5}))
    

# pd.concat(preds1)[['ID', 'Close']].to_csv('result.csv', index=False)

pd.concat(preds)[['ID', 'Close']].to_csv('result1.csv', index=False)
# pd.concat(preds1).shape       

In [ ]:
pd.concat(preds)['Score'].unique()

In [ ]:
pd.concat(preds)['Params'].unique()

In [ ]:
# FileLink('result.csv')

In [ ]:
FileLink('result1.csv')

In [ ]:

# !cat result.csv

In [ ]:
# for stock in X_test.stock.unique():
!cat result1.csv

In [ ]:
pd.concat(preds)['Params'].unique()

In [ ]:
pd.concat(preds).to_csv("result_xgboost_analysis.csv", index=False)
FileLink('result_xgboost_analysis.csv')